<a href="https://colab.research.google.com/github/woo2201/sampleCases/blob/reservior/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
cd /content/gdrive/MyDrive/dataset/April/

/content/gdrive/MyDrive/dataset/April


In [4]:
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import GlobalMaxPooling1D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Concatenate, LSTM, GRU
from tensorflow.keras.layers import Bidirectional, Multiply

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import roc_auc_score

from sklearn.model_selection import KFold, GroupKFold

In [5]:
import numpy as np
import scipy as sp
import pandas as pd
import os
import glob

path = "/content/gdrive/MyDrive/dataset/April/"
filenames = glob.glob(path + "*.csv")
data = [var.split('.')[0].split('/')[6] for var in filenames]

# dynamically allocate df names.
for name in zip(filenames, data):
    loc = list(name)[0]
    val = list(name)[1]
    globals()[val] = pd.read_csv(loc)

In [39]:
train

,sequence,subject,step,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09,sensor_10,sensor_11,sensor_12
0,0,47,0,-0.196291,0.112395,1.000000,0.329204,-1.004660,-0.131638,-0.127505,0.368702,-0.1,-0.963873,-0.985069,0.531893,4.751492
1,0,47,1,-0.447450,0.134454,1.000000,-0.658407,0.162495,0.340314,-0.209472,-0.867176,0.2,-0.301301,0.082733,-0.231481,0.454390
2,0,47,2,0.326893,-0.694328,1.000000,0.330088,0.473678,1.280479,-0.094718,0.535878,1.4,1.002168,0.449221,-0.586420,-4.736147
3,0,47,3,0.523184,0.751050,1.000000,0.976991,-0.563287,-0.720269,0.793260,0.951145,-0.3,-0.995665,-0.434290,1.344650,0.429241
4,0,47,4,0.272025,1.074580,1.000000,-0.136283,0.398579,0.044877,0.560109,-0.541985,-0.9,1.055636,0.812631,0.123457,-0.223359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1558075,25967,327,55,-0.282844,-1.217437,-1.666153,0.586726,-0.930698,-0.451010,-0.651184,0.368702,0.4,0.008671,-0.723536,-0.353909,-0.914749
1558076,25967,327,56,0.130603,0.349790,-1.666153,-0.324779,0.775324,-0.332835,0.099271,0.122137,-0.2,0.644509,0.691407,-0.613169,-0.515772
1558077,25967,327,57,-0.579598,0.429622,-1.666153,0.319469,0.308861,0.282723,-0.512750,0.012214,-1.6,-0.424133,0.716855,1.628601,0.928389
1558078,25967,327,58,1.278980,1.711134,-1.522820,0.802655,-0.460541,-0.055348,2.405282,0.043511,1.9,0.283960,-0.914914,0.364198,0.211424


In [44]:
train.columns.tolist()[3:]

['sensor_00',
 'sensor_01',
 'sensor_02',
 'sensor_03',
 'sensor_04',
 'sensor_05',
 'sensor_06',
 'sensor_07',
 'sensor_08',
 'sensor_09',
 'sensor_10',
 'sensor_11',
 'sensor_12']

In [6]:
features = train.columns.tolist()[3:] # bring the sensor-columns.

def prep(df):
    for feature in features:
        df[feature + '_lag1'] = df.groupby('sequence')[feature].shift(1) # make a lag_1 column
        df.fillna(0, inplace=True)
        df[feature + '_diff1'] = df[feature] - df[feature + '_lag1']    # 각 features 들에 대해서 - lag 컬럼과의 차이를 구해준다.

In [7]:
prep(train)
prep(test) # inplace = True 기 때문에 굳이 그렇게 안해줘도 된다.

In [8]:
features = train.columns.tolist()[3:]
sc = StandardScaler()
train[features] = sc.fit_transform(train[features])
test[features] = sc.transform(test[features])

In [9]:
groups = train["sequence"]
labels = train_labels["state"]

train = train.drop(["sequence", "subject", "step"], axis=1).values
train = train.reshape(-1, 60, train.shape[-1])

test = test.drop(["sequence", "subject", "step"], axis=1).values
test = test.reshape(-1, 60, test.shape[-1])

In [10]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # https://blog.pingpong.us/tpu-with-tf2-and-gcp/
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
    BATCH_SIZE = tpu_strategy.num_replicas_in_sync * 64
    print("Running on TPU:", tpu.master())
    print(f"Batch Size: {BATCH_SIZE}")
    
except ValueError:
    strategy = tf.distribute.get_strategy()
    BATCH_SIZE = 256
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    print(f"Batch Size: {BATCH_SIZE}")
    
    
def dnn_model():
    
    x_input = Input(shape=(train.shape[-2:]))
    
    x1 = Bidirectional(LSTM(units=512, return_sequences=True))(x_input)
    x2 = Bidirectional(LSTM(units=256, return_sequences=True))(x1)
    z1 = Bidirectional(GRU(units=256, return_sequences=True))(x1)
    
    c = Concatenate(axis=2)([x2, z1])
    
    x3 = Bidirectional(LSTM(units=128, return_sequences=True))(c)
    
    x4 = GlobalMaxPooling1D()(x3)
    x5 = Dense(units=128, activation='selu')(x4)
    x_output = Dense(1, activation='sigmoid')(x5)

    model = Model(inputs=x_input, outputs=x_output, name='lstm_model')
    
    return model

model = dnn_model()

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.87.81.122:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.87.81.122:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Running on TPU: grpc://10.87.81.122:8470
Batch Size: 512


In [11]:
with tpu_strategy.scope():

    VERBOSE = True
    predictions, scores = [], []
    k = GroupKFold(n_splits = 10)

    for fold, (train_idx, val_idx) in enumerate(k.split(train, labels, groups.unique())):

        print('-'*15, '>', f'Fold {fold+1}', '<', '-'*15)
    
        X_train, X_val = train[train_idx], train[val_idx]
        y_train, y_val = labels.iloc[train_idx].values, labels.iloc[val_idx].values
        
        model = dnn_model()
        model.compile(optimizer="adam",
                      loss="binary_crossentropy",
                      metrics='AUC')

        lr = ReduceLROnPlateau(monitor="val_auc",
                               factor=0.6, 
                               patience=4,
                               verbose=VERBOSE) # 모델의 개선이 없을 경우, Learning Rate를 조절해 모델의 개선을 유도하는 콜백함수입니다. # https://deep-deep-deep.tistory.com/56

        es = EarlyStopping(monitor="val_auc",
                           patience=7, 
                           verbose=VERBOSE,
                           mode="max", 
                           restore_best_weights=True)
        
        save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
        chk_point = ModelCheckpoint(f'./TPS_model_2022_{fold+1}C.h5',
                                    options=save_locally, 
                                    monitor='val_auc',
                                    verbose=VERBOSE, 
                                    save_best_only=True,
                                    mode='max')
        
        model.fit(X_train, y_train, 
                  validation_data = (X_val, y_val), 
                  epochs = 15,
                  verbose = VERBOSE,
                  batch_size = BATCH_SIZE, 
                  callbacks = [lr, chk_point, es])
        
        load_locally = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
        model = load_model(f'./TPS_model_2022_{fold+1}C.h5', options=load_locally)
        
        y_pred = model.predict(X_val, batch_size=BATCH_SIZE).squeeze()
        score = roc_auc_score(y_val, y_pred)
        scores.append(score)
        predictions.append(model.predict(test, batch_size=BATCH_SIZE).squeeze())
        print(f"Fold-{fold+1} | OOF Score: {score}")
    
    print(f'Mean accuracy on {k.n_splits} folds - {np.mean(scores)}')

--------------- > Fold 1 < ---------------
Epoch 1/15
46/46 [==============================] - ETA: 0s - loss: 0.5499 - auc: 0.7960
Epoch 1: val_auc improved from -inf to 0.86375, saving model to ./TPS_model_2022_1C.h5
46/46 [==============================] - 50s 614ms/step - loss: 0.5499 - auc: 0.7960 - val_loss: 0.4634 - val_auc: 0.8637 - lr: 0.0010
Epoch 2/15
46/46 [==============================] - ETA: 0s - loss: 0.3717 - auc: 0.9136
Epoch 2: val_auc improved from 0.86375 to 0.91555, saving model to ./TPS_model_2022_1C.h5
46/46 [==============================] - 10s 221ms/step - loss: 0.3717 - auc: 0.9136 - val_loss: 0.4733 - val_auc: 0.9156 - lr: 0.0010
Epoch 3/15
46/46 [==============================] - ETA: 0s - loss: 0.3157 - auc: 0.9387
Epoch 3: val_auc improved from 0.91555 to 0.93907, saving model to ./TPS_model_2022_1C.h5
46/46 [==============================] - 10s 217ms/step - loss: 0.3157 - auc: 0.9387 - val_loss: 0.3324 - val_auc: 0.9391 - lr: 0.0010
Epoch 4/15
46/46 [

In [23]:
sample_submission["state"] = sum(predictions)/k.n_splits

In [25]:
sample_submission.to_csv('submission_1.csv', index=False)